# Crawling
- 동적으로 서울, 인천, 경기의 각 구에 대하여 업소, 유동인구, 매출 추출

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time
driver = webdriver.Chrome()
driver.get("https://sg.sbiz.or.kr/godo/index.sg#") 

In [15]:
data_list = []

# 서울, 인천, 경기 코드 하드코딩
regions = {"서울": "11", "인천": "28", "경기": "41"}

# 버튼 코드 하드코딩
categories = {
    "업소": 1,
    "유동인구": 2,
    "매출": 3
}

for region_name, region_code in regions.items():

    # 서울, 인천, 경기 선택
    try:
        main_select = Select(driver.find_element(By.ID, "rnk_select_mega"))
        main_select.select_by_value(region_code)
        time.sleep(2) 
    except Exception as e:
        print(f"Error selecting region {region_name}: {e}")
        continue

    # 행정구 options 추출
    try:
        sub_select = Select(driver.find_element(By.ID, "rnk_select_sgg"))
        sub_options = sub_select.options
    except Exception as e:
        print(f"Error finding sub-select options for region {region_name}: {e}")
        continue

    # 행정구 loop
    for option in sub_options[1:]:  # '전체' 키워드 제외
        try:
            sub_select.select_by_value(option.get_attribute("value"))
            time.sleep(2)  
        except Exception as e:
            print(f"Error selecting sub-region {option.text} for region {region_name}: {e}")
            continue

        # category 중 하나 선택 -> 값 추출
        for category, type_id in categories.items():
            try:
                driver.find_element(By.CSS_SELECTOR, f"a[data-type='{type_id}']").click()
                time.sleep(2) 
            except Exception as e:
                print(f"Error selecting category {category} for region {region_name}: {e}")
                continue

            # HTML Parsing
            try:
                html_data = driver.page_source
                soup = BeautifulSoup(html_data, 'html.parser')

                for row in soup.select('#dynamicTbody tr'):
                    tds = row.find_all('td')
                    if len(tds) < 3:
                        continue
                    region_name = tds[1].get_text(separator=">").replace('&gt;', '>').strip()
                    category_value = tds[2].get_text().replace(',', '').strip()
                    data_list.append((region_name, category, int(category_value)))
                    
            except Exception as e:
                print(f"Error parsing data for category {category} in region {region_name}: {e}")
                continue

try:
    df = pd.DataFrame(data_list, columns=["Region", "Category", "Value"])
    # df.to_csv("Data/행정동_업소_유동인구_매출_서울_경기도_인천_구별.csv", index=False)
    print(df.head())
    print(f"Length of data_list: {len(data_list)}")
except Exception as e:
    print(f"Error saving data to CSV: {e}")


           Region Category  Value
0  서울특별시>강남구>역삼1동       업소  38078
1  서울특별시>강남구>압구정동       업소   9183
2   서울특별시>강남구>신사동       업소   9313
3   서울특별시>강남구>청담동       업소  11620
4  서울특별시>강남구>논현1동       업소  13300
Length of data_list: 3451


In [20]:
df_pivot = df.pivot_table(index="Region", columns="Category", values="Value").reset_index()

df_pivot.columns = ["Address", "업소수", "매출", "유동인구"]

df_pivot["시"] = df_pivot["Address"].str.split(">").str[0]
df_pivot["구"] = df_pivot["Address"].str.split(">").str[1]
df_pivot["동"] = df_pivot["Address"].str.split(">").str[2]
df_pivot["Address"] = df_pivot["Address"].str.replace(">", " ")

df_pivot.drop_duplicates()

display(df_pivot)

df_pivot.to_csv("Data/행정동_업소_유동인구_매출_서울_경기도_인천_구별.csv", index=False, encoding="cp949")

,Address,업소수,매출,유동인구,시,구,동
0,경기도 가평군 가평읍,3212567.0,4018.0,136677.0,경기도,가평군,가평읍
1,경기도 가평군 북면,436316.0,1006.0,30605.0,경기도,가평군,북면
2,경기도 가평군 상면,1166761.0,1468.0,40909.0,경기도,가평군,상면
3,경기도 가평군 설악면,3700145.0,2316.0,139088.0,경기도,가평군,설악면
4,경기도 가평군 조종면,1512576.0,1603.0,36304.0,경기도,가평군,조종면
...,...,...,...,...,...,...,...
1146,인천광역시 중구 영종2동,2397693.0,7045.0,122714.0,인천광역시,중구,영종2동
1147,인천광역시 중구 영종동,2311382.0,3885.0,372414.0,인천광역시,중구,영종동
1148,인천광역시 중구 용유동,3835020.0,1761.0,153972.0,인천광역시,중구,용유동
1149,인천광역시 중구 운서동,12671576.0,10229.0,710715.0,인천광역시,중구,운서동
